<a href="https://colab.research.google.com/github/pkant-0/duplicate_records/blob/main/handle_duplicate_records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I am sharing this colab notebook.
You can use it and leave your comments on the steps which is not working.
any suggestion or change.

In [1]:
%%capture
!pip install pandas numpy recordlinkage fuzzywuzzy

In [2]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import recordlinkage
from fuzzywuzzy import process

# Set random seed for reproducibility
np.random.seed(42)

# Parameters
num_records = 1000  # Total records
num_duplicates = 200  # Number of duplicate records
base_time = datetime(2022, 1, 1)

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# Generate synthetic multimedia data
data = []
for i in range(num_records):
    # Create a base record
    title = f"Media_Item_{i+1}"
    file_type = random.choice(['image', 'video', 'audio'])
    last_modified = base_time + timedelta(days=random.randint(0, 365))
    size = random.randint(1, 100)  # Size in MB
    # Add a potential duplicate record
    if i < num_duplicates:
        duplicate = random.choice(range(num_records))  # Randomly link to an existing record
        title = f"Media_Item_{duplicate+1}"  # Duplicate title
        last_modified = base_time + timedelta(days=random.randint(0, 365))  # Random last modified date

    data.append({
        'ID': i + 1,
        'Title': title,
        'FileType': file_type,
        'LastModified': last_modified,
        'Size_MB': size
    })

In [5]:
# Create DataFrame and save it as CSV
df = pd.DataFrame(data)
df.to_csv("multimedia_data.csv", index=False)
print("CSV file 'multimedia_data.csv' created.")

CSV file 'multimedia_data.csv' created.


In [6]:
df.head()

,ID,Title,FileType,LastModified,Size_MB
0,1,Media_Item_471,audio,2022-03-10,42
1,2,Media_Item_577,audio,2022-03-23,4
2,3,Media_Item_778,image,2022-08-22,2
3,4,Media_Item_701,audio,2022-03-31,49
4,5,Media_Item_396,video,2022-07-06,43


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            1000 non-null   int64         
 1   Title         1000 non-null   object        
 2   FileType      1000 non-null   object        
 3   LastModified  1000 non-null   datetime64[ns]
 4   Size_MB       1000 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 39.2+ KB


In [9]:
# Load the data from CSV file
df = pd.read_csv("multimedia_data.csv", parse_dates=['LastModified'])
print("Data loaded from 'multimedia_data.csv':")
print(df.head())

Data loaded from 'multimedia_data.csv':
   ID           Title FileType LastModified  Size_MB
0   1  Media_Item_471    audio   2022-03-10       42
1   2  Media_Item_577    audio   2022-03-23        4
2   3  Media_Item_778    image   2022-08-22        2
3   4  Media_Item_701    audio   2022-03-31       49
4   5  Media_Item_396    video   2022-07-06       43


In [10]:
# Identifying exact duplicates
duplicates = df[df.duplicated(subset='Title', keep=False)]
print("Exact Duplicates:")
print(duplicates)

Exact Duplicates:
      ID           Title FileType LastModified  Size_MB
0      1  Media_Item_471    audio   2022-03-10       42
1      2  Media_Item_577    audio   2022-03-23        4
2      3  Media_Item_778    image   2022-08-22        2
3      4  Media_Item_701    audio   2022-03-31       49
4      5  Media_Item_396    video   2022-07-06       43
..   ...             ...      ...          ...      ...
978  979  Media_Item_979    audio   2022-08-06       30
981  982  Media_Item_982    video   2022-07-22       83
982  983  Media_Item_983    image   2022-09-20       48
983  984  Media_Item_984    video   2022-07-25       71
994  995  Media_Item_995    video   2022-09-19       52

[301 rows x 5 columns]


In [11]:
# Create a record linkage index
indexer = recordlinkage.Index()
indexer.block('FileType')  # Block on file type to reduce comparisons

<Index>

In [12]:
# Create pairs for comparison
candidate_links = indexer.index(df)

In [13]:
# Define comparison features
compare = recordlinkage.Compare()

compare.string('Title', 'Title', method='jarowinkler', label='Title')
compare.numeric('Size_MB', 'Size_MB', label='Size')
compare.date('LastModified', 'LastModified', label='LastModified')

<Compare>

In [14]:
# Compute comparison vectors
features = compare.compute(candidate_links, df)

/usr/local/lib/python3.10/dist-packages/recordlinkage/compare.py:388: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  c[


In [16]:
# Classify links based on similarity scores
matches = features[features.sum(axis=1) > 1]  # Keeping pairs with similarity greater than 1

In [17]:
print("Probabilistic Matches:")
print(matches)

Probabilistic Matches:
            Title  Size  LastModified
14  13   0.914286   0.5           0.0
20  11   0.914286   1.0           0.0
22  17   0.942857   0.5           0.0
23  17   0.942857   1.0           0.0
    22   0.914286   0.5           0.0
...           ...   ...           ...
999 629  0.931429   0.0           1.0
    837  0.903810   1.0           0.0
    853  0.903810   1.0           0.0
    928  0.903810   0.5           0.0
    951  0.903810   1.0           0.0

[5714 rows x 3 columns]
